In [ ]:
#Bibliotecas usadas no projeto
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [ ]:
items = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/questao_prova_mapping.csv")
items["TP_LINGUA"] = items["TP_LINGUA"].map({0: 'english', 1: 'spanish', np.nan: 'portuguese'})
codes = {'CN':1092,'CH':1062,'LC':1072,'MT':1082}
laranja = items[items["CO_PROVA"].isin(codes.values())].sort_values(by="CO_POSICAO")
laranja = laranja[["CO_POSICAO","CO_ITEM","SG_AREA","TP_LINGUA","IN_ITEM_ABAN","NU_PARAM_A","NU_PARAM_B","NU_PARAM_C"]].drop_duplicates()


In [ ]:
gpt4MT = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT4/solution_enem_22_mt.csv")
gpt4CH = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT4/solution_enem_22_ch.csv")
gpt4CN = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT4/solution_enem_22_cn.csv")
gpt4LC = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT4/solution_enem_22_lc.csv")
gpt4LC.head(2)

# Concatenate DataFrames vertically
gpt4 = pd.concat([gpt4MT, gpt4CH, gpt4CN, gpt4LC], ignore_index=True).sort_values(by='question')[["question_number","test","language","gpt-4_probs","answer"]]
# Display the combined DataFrame
gpt4

,question_number,test,language,gpt-4_probs,answer
140,1,LC,spanish,"{'A': 0.9997000567012279, 'E': 3.3255359580266...",A
135,1,LC,english,"{'B': 0.9997995716248057, 'D': 2.8285662100873...",B
136,2,LC,english,"{'D': 0.9998999808033897, 'B': 1.0798592535453...",D
141,2,LC,spanish,"{'A': 0.9996999562911968, 'B': 1.3359866929778...",A
142,3,LC,spanish,"{'D': 0.9998999968803092, 'A': 7.0818900192959...",D
...,...,...,...,...,...
94,95,CN,portuguese,"{'D': 0.9999998096227547, 'A': 1.0907283173840...",D
95,96,CN,portuguese,"{'B': 0.9998687449638521, 'C': 3.0238907245606...",B
96,97,CN,portuguese,"{'E': 0.9997998368503119, 'D': 1.5620827375503...",E
97,98,CN,portuguese,"{'D': 0.6186411498214642, 'B': 0.1828699691023...",A


In [ ]:
import json
def dist_to_probs(x):
    probs = [0,0,0,0,0]
    probs[0] = x['A']
    probs[1] = x['B']
    probs[2] = x['C']
    probs[3] = x['D']
    probs[4] = x['E']
    probs = np.array(probs)
    remain = (1.0 - np.sum(probs))/len(probs)
    probs = probs + remain
    return probs

gpt4['gpt-4_probs'] = gpt4['gpt-4_probs'].apply(lambda x: json.loads(x.replace('\'','\"')))
gpt4['model_probs']= gpt4['gpt-4_probs'].apply(dist_to_probs)

In [ ]:
gpt4 = pd.merge(gpt4, laranja, how='left', left_on=['question_number',"language"], right_on=['CO_POSICAO',"TP_LINGUA"])

In [ ]:
gpt4 = gpt4.drop(columns=['question_number', 'test',"gpt-4_probs"])
gpt4["model"] = "GPT4"
gpt4["is_human"] = False

In [ ]:
gpt4.head()

,language,answer,model_probs,CO_POSICAO,CO_ITEM,SG_AREA,TP_LINGUA,IN_ITEM_ABAN,NU_PARAM_A,NU_PARAM_B,NU_PARAM_C,model,is_human
0,spanish,A,"[0.9997000567012279, 9.997001447087808e-05, 9....",1,140567,LC,spanish,0,1.95054,-0.11616,0.18874,GPT4,False
1,english,B,"[1.8550190097388833e-07, 0.9997995716248057, 9...",1,140767,LC,english,0,4.34313,0.36871,0.44865,GPT4,False
2,english,D,"[9.987065490819526e-09, 1.079859253545349e-08,...",2,140731,LC,english,0,2.88489,0.49591,0.18224,GPT4,False
3,spanish,A,"[0.9996999562911968, 1.3359866925337124e-07, 9...",2,140527,LC,spanish,0,1.12895,-0.11509,0.20583,GPT4,False
4,spanish,D,"[7.0818900192959935e-09, 9.999000099990002e-05...",3,118252,LC,spanish,0,1.93731,0.91995,0.14292,GPT4,False


In [ ]:
gpt3MT = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT3/solution_enem_22_mt.csv")
gpt3CH = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT3/solution_enem_22_ch.csv")
gpt3CN = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT3/solution_enem_22_cn.csv")
gpt3LC = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GPT/SolvedPortugueseGPT3/solution_enem_22_lc.csv")
gpt3LC.head(2)

# Concatenate DataFrames vertically
gpt3 = pd.concat([gpt3MT, gpt3CH, gpt3CN, gpt3LC], ignore_index=True).sort_values(by='question')[["question_number","test","language","gpt-3.5-turbo_probs","answer"]]
# Display the combined DataFrame
gpt3

,question_number,test,language,gpt-3.5-turbo_probs,answer
140,1,LC,spanish,"{'A': 0.9723202498709326, 'C': 0.0001, 'B': 0....",A
135,1,LC,english,"{'B': 0.9960978914233115, 'A': 0.0017905043386...",B
136,2,LC,english,"{'D': 0.9776260944016127, 'A': 0.0053523267685...",D
141,2,LC,spanish,"{'A': 0.8839154066472396, 'B': 0.0091584456389...",A
142,3,LC,spanish,"{'D': 0.9825035437071137, 'A': 0.0033886767080...",D
...,...,...,...,...,...
94,95,CN,portuguese,"{'A': 0.5406801435461193, 'C': 0.2099474732621...",D
95,96,CN,portuguese,"{'B': 0.8866408823771144, 'A': 0.0931771698188...",B
96,97,CN,portuguese,"{'E': 0.8806321367236951, 'A': 0.0965663953794...",E
97,98,CN,portuguese,"{'A': 0.3782197730982776, 'C': 0.1899600595141...",A


In [ ]:
gpt3['gpt-3.5-turbo_probs'] = gpt3['gpt-3.5-turbo_probs'].apply(lambda x: json.loads(x.replace('\'','\"')))
gpt3['model_probs']= gpt3['gpt-3.5-turbo_probs'].apply(dist_to_probs)

In [ ]:
gpt3 = pd.merge(gpt3, laranja, how='left', left_on=['question_number',"language"], right_on=['CO_POSICAO',"TP_LINGUA"])

In [ ]:
gpt3 = gpt3.drop(columns=['question_number', 'test',"gpt-3.5-turbo_probs"])
gpt3["model"] = "GPT3"
gpt3["is_human"] = False


In [ ]:
gpt3.head()

,language,answer,model_probs,CO_POSICAO,CO_ITEM,SG_AREA,TP_LINGUA,IN_ITEM_ABAN,NU_PARAM_A,NU_PARAM_B,NU_PARAM_C,model,is_human
0,spanish,A,"[0.977776199896746, 0.005555950025813494, 0.00...",1,140567,LC,spanish,0,1.95054,-0.11616,0.18874,GPT3,False
1,english,B,"[0.0020747769170867537, 0.9963821640017246, 0....",1,140767,LC,english,0,4.34313,0.36871,0.44865,GPT3,False
2,english,D,"[0.008696642534513705, 0.0034443157659683863, ...",2,140731,LC,english,0,2.88489,0.49591,0.18224,GPT3,False
3,spanish,A,"[0.9048617436167011, 0.030104782608454055, 0.0...",2,140527,LC,spanish,0,1.12895,-0.11509,0.20583,GPT3,False
4,spanish,D,"[0.006150232624978823, 0.0028615559169768655, ...",3,118252,LC,spanish,0,1.93731,0.91995,0.14292,GPT3,False


In [ ]:
mariMT = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/Solved_Maritalk/solved_maritalk_new_prompt/Respostas_maritalk_laranja_2022_new_prompt_MT.csv")
mariCH = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/Solved_Maritalk/solved_maritalk_new_prompt/Respostas_maritalk_laranja_2022_new_prompt_CH.csv")
mariCN = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/Solved_Maritalk/solved_maritalk_new_prompt/Respostas_maritalk_laranja_2022_new_prompt_CN.csv")
mariLC = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/Solved_Maritalk/solved_maritalk_new_prompt/Respostas_maritalk_laranja_2022_new_prompt_LC.csv")
# Concatenate DataFrames vertically
maritalk = pd.concat([mariMT, mariCH, mariCN, mariLC], ignore_index=True).sort_values(by='questao')
maritalk = pd.merge(maritalk, laranja, how='left', left_on="questao",right_on="CO_POSICAO")

maritalk["model_probs"] = list(maritalk[["A","B","C","D","E"]].values)

# maritalk["model_probs"] = maritalk["model_probs"] + ((1 - maritalk["model_probs"].apply(np.sum))/5)

# maritalk["model_probs"].apply(np.sum)
maritalk["model"] = "MariTalk"
maritalk["is_human"] = False
# maritalk = maritalk.rename(columns={'probability': 'model_probs'})

# Display the combined DataFrame
maritalk

,questao,idioma,gabarito,resposta,A,B,C,D,E,NA,...,CO_ITEM,SG_AREA,TP_LINGUA,IN_ITEM_ABAN,NU_PARAM_A,NU_PARAM_B,NU_PARAM_C,model_probs,model,is_human
0,1,english,B,B,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,140767,LC,english,0,4.34313,0.36871,0.44865,"[0.0, 1.0, 0.0, 0.0, 0.0]",MariTalk,False
1,1,english,B,B,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,140567,LC,spanish,0,1.95054,-0.11616,0.18874,"[0.0, 1.0, 0.0, 0.0, 0.0]",MariTalk,False
2,1,spanish,A,A,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,140767,LC,english,0,4.34313,0.36871,0.44865,"[1.0, 0.0, 0.0, 0.0, 0.0]",MariTalk,False
3,1,spanish,A,A,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,140567,LC,spanish,0,1.95054,-0.11616,0.18874,"[1.0, 0.0, 0.0, 0.0, 0.0]",MariTalk,False
4,2,spanish,A,A,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,140731,LC,english,0,2.88489,0.49591,0.18224,"[1.0, 0.0, 0.0, 0.0, 0.0]",MariTalk,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,176,portuguese,B,B,0.174757,0.427184,0.320388,0.058252,0.000000,0.019417,...,10500,MT,portuguese,0,1.57712,3.30111,0.16855,"[0.174757281553398, 0.4271844660194174, 0.3203...",MariTalk,False
191,177,portuguese,E,A,0.533981,0.019417,0.223301,0.077670,0.038835,0.106796,...,89637,MT,portuguese,1,NaN,NaN,NaN,"[0.5339805825242718, 0.0194174757281553, 0.223...",MariTalk,False
192,178,portuguese,E,C,0.077670,0.155340,0.601942,0.097087,0.067961,0.000000,...,97590,MT,portuguese,0,2.33252,2.40899,0.14217,"[0.0776699029126213, 0.1553398058252427, 0.601...",MariTalk,False
193,179,portuguese,C,A,0.475728,0.213592,0.087379,0.048544,0.058252,0.116505,...,86840,MT,portuguese,0,0.24441,4.65334,0.03228,"[0.4757281553398058, 0.2135922330097087, 0.087...",MariTalk,False


In [ ]:
url = "https://raw.githubusercontent.com/marceloslo/EnemParser/main/QuestionDistributionGenerator/DISTRIBUTIONS/answer_dist_per_question_NSE_LEVEL.csv"
nse = pd.read_csv(url)
nse

,CO_ITEM,NSE_LEVEL,TX_GABARITO,answer_distribution,correct_probability
0,5961,1,D,[0.17715449 0.24600769 0.19172133 0.23854332 0...,0.238543
1,5961,2,D,[0.18086618 0.23935467 0.18306477 0.25156946 0...,0.251569
2,5961,3,D,[0.18476724 0.22741081 0.17515537 0.26701003 0...,0.267010
3,5961,4,D,[0.19089704 0.21614213 0.16632591 0.28207953 0...,0.282080
4,5961,5,D,[0.19956792 0.20120938 0.15833383 0.30038837 0...,0.300388
...,...,...,...,...,...
1555,141731,4,E,[0.26186 0.36858586 0.10159851 0.14330812 0...,0.122329
1556,141731,5,E,[0.27796015 0.34116765 0.11130476 0.1455975 0...,0.121810
1557,141731,6,E,[0.29243962 0.30876561 0.12821715 0.14397896 0...,0.124384
1558,141731,7,E,[0.30766613 0.26650759 0.152791 0.13631305 0...,0.134405


In [ ]:
# Mapping for race/ethnicity categories
mapping_SES = {
    0: "SES-1",
    1: "SES-1",
    2: "SES-2",
    3: "SES-3",
    4: "SES-4",
    5: "SES-5",
    6: "SES-6",
    7: "SES-7",
    8: "SES-8",
}
# Map the 'Ethnicity' column using the defined mapping
nse['NSE_LEVEL'] = nse['NSE_LEVEL'].map(mapping_SES)
nse.rename(columns={'NSE_LEVEL': 'NSE_Level'}, inplace=True)

In [ ]:
import ast
# Replace ' ' with ',' and fix the last element in each list
nse['answer_distribution'] = nse['answer_distribution'].apply(lambda x: ",".join(x.split()).replace(",]", "]"))
# Convert string representations to actual lists
nse['answer_distribution'] = nse['answer_distribution'].apply(ast.literal_eval)
nse['answer_distribution'] = nse['answer_distribution'].apply(lambda x: np.array(x[:-1]))
nse['answer_distribution'][0]

array([0.17715449, 0.24600769, 0.19172133, 0.23854332, 0.14259217])

In [ ]:
nse = nse.rename(columns={'answer_distribution': 'model_probs'})

In [ ]:
# nse["model_probs"] = nse["model_probs"] + ((1 - nse["model_probs"].apply(np.sum))/5)

In [ ]:
nse = pd.merge(nse, laranja, how='left', on="CO_ITEM")
nse["model"] = nse["NSE_Level"]
nse["is_human"] = True


In [ ]:
nse.dropna(inplace=True)

In [ ]:
columns = ["CO_ITEM","SG_AREA","TP_LINGUA","IN_ITEM_ABAN","model","is_human","model_probs","NU_PARAM_A","NU_PARAM_B","NU_PARAM_C"]
models = pd.concat([gpt4[columns],gpt3[columns],maritalk[columns],nse[columns]]).reset_index()

In [ ]:
models = models[models["IN_ITEM_ABAN"] == 0]

In [ ]:
models.head()

,index,CO_ITEM,SG_AREA,TP_LINGUA,IN_ITEM_ABAN,model,is_human,model_probs,NU_PARAM_A,NU_PARAM_B,NU_PARAM_C
0,0,140567,LC,spanish,0.0,GPT4,False,"[0.9997000567012279, 9.997001447087808e-05, 9....",1.95054,-0.11616,0.18874
1,1,140767,LC,english,0.0,GPT4,False,"[1.8550190097388833e-07, 0.9997995716248057, 9...",4.34313,0.36871,0.44865
2,2,140731,LC,english,0.0,GPT4,False,"[9.987065490819526e-09, 1.079859253545349e-08,...",2.88489,0.49591,0.18224
3,3,140527,LC,spanish,0.0,GPT4,False,"[0.9996999562911968, 1.3359866925337124e-07, 9...",1.12895,-0.11509,0.20583
4,4,118252,LC,spanish,0.0,GPT4,False,"[7.0818900192959935e-09, 9.999000099990002e-05...",1.93731,0.91995,0.14292


In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GabaritosCsv/2023_Dia_1_P1_Caderno_09_Laranja_NVDA.csv")
df = pd.concat([df,pd.read_csv("https://raw.githubusercontent.com/marceloslo/EnemParser/main/GabaritosCsv/2023_Dia_2_P1_Caderno_11_Laranja_NVDA.csv")])
df["test"] = df["test"].fillna("portuguese")
df.head()

,question,answer,test
0,1,B,english
1,1,B,spanish
2,2,A,spanish
3,2,D,english
4,3,A,spanish


In [ ]:
merged_df = pd.merge(df, laranja, left_on=["question","test"], right_on=["CO_POSICAO","TP_LINGUA"],how='inner')[["CO_ITEM","answer"]]
merged_df.head()

,CO_ITEM,answer
0,140767,B
1,140567,B
2,140527,A
3,140731,D
4,118252,A


In [ ]:
models = pd.merge(models,merged_df,on="CO_ITEM",how="left")

In [ ]:
models = models[models["answer"] != "anulada"]

In [ ]:
number_map = {letter: index for index, letter in enumerate('ABCDE')}

def get_model_prob(row):
    index = number_map[row['answer']]
    return row['model_probs'][index]

models['correct_prob'] = models.apply(get_model_prob, axis=1)

In [ ]:
models.to_csv("models_cleanned.csv",index=False)